In [ ]:
import pandas as pd
import tweepy
from tweepy import API 
from tweepy import OAuthHandler
import json

In [ ]:
#empty lists to store data

#"tweet_coord",
#"tweet_location","user_timezone"  
columns_names=[]
created_at=[] #"tweet_created"
id=[] # "tweet_id" 
full_text=[] #"text"
entities_hashtag=[] 
user_name=[] #"name"
user_screen_name=[] #name
user_location=[] #"tweet_location"
coordinates=[] #"tweet_coord"
retweet_count=[]#"retweet_count"
lang=[] 
user_acc_date=[]
user_verified=[]
full_retweet=[]

#these are the phrases we will be searching for with the twitter API 
air_search_parameter=['@united','@AmericanAir','@Delta','@SouthwestAir','@FlyFrontier',  
                      '@SpiritAirlines','@SunCountryAir','@AlaskaAir','@CapeAir',
                      '@HawaiianAir','@VirginAmerica','@JetBlue','@USAirways']

airline_list=['United','American','Delta','Southwest','Fly Frontier',
              'Spirit Airlines','Sun Country Air','Alaska Air','Cape Air',
              'Haiwaiin Air','Virgin America','Jet Blue','US Airways']

airlines=[]

In [ ]:
#Setup access to API, please note that you would need to enter your own personal keys/tokens 
# API keys have to be requested from twitter on your account
twitter_keys = {
        'consumer_key':        '######################',
        'consumer_secret':     '######################',
        'access_token_key':    '######################',
        'access_token_secret': '#######################'
    }

auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

# only ~15,0000 tweets can be gathered every 15 minutes
# We set wait_on_rate_limit to True in order to avoid timing out. 
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [ ]:
ticker=0  # allows us to keep track of which airline the tweet is directed at
i=0       # keeps track of tweets 

for q_search in air_search_parameter:     # loop that goes through airline 
    airline=airline_list[ticker]          # variable to keep track of airline 
    
    # each "page" is a collection of ~100 or so tweets gathered at once from the api 
    # q=q_search parameter tells the api what phrase to search for, 
    # an example would be '@united' which would return all tweets containing the string '@united'
    # tweet_mode=extended allows us to extract text even if it is a retweet. 
    # Please note that you can only search as far back as a week
    for page in tweepy.Cursor(api.search, q=q_search, count=100,since="2020-11-25",              
                               until="2020-12-02",lang='en', tweet_mode='extended').pages(1000):
        
        for k in range(0,len(page)):  # loop through every tweet in the page
            
            #each tweet comes in a json format, and we extract the relevant variables from it 
            try: 
                status=page[k]._json   
                created_at.append(status['created_at']) #"tweet_created"
                id.append(status['id']) # "tweet_id" 
                full_text.append(status['full_text']) #"text"
                t_entities_hashtag=status['entities']['hashtags'] # possibly important
                hashtags=[] #possibly important
                for k in range(0,len(t_entities_hashtag)):
                    hashtags.append(t_entities_hashtag[k]['text'])
                entities_hashtag.append(hashtags)
                user_name.append(status['user']['name']) #"name"
                user_screen_name.append(status['user']['screen_name']) #"user_name"
                user_location.append(status['user']['location'])#"tweet_location"
                user_acc_date.append(status['user']['created_at']) 
                user_verified.append(status['user']['verified'])
                coordinates.append(status['coordinates']) #"tweet_coord"
                airlines.append(airline)
                retweet_count.append(status['retweet_count']) #"retweet_count"
                lang.append(status['lang'])
                i=i+1
            except:
                continue
                
        print(str(i)+' tweets collected so far') 
    print(q_search)
    print(airline)
    print('FINISHED')
    ticker=ticker+1                             #move on to the next airline 

In [ ]:
#creates dataframe from columns using pandas

columns = {'tweet_created':created_at,  #"tweet_created"
        'tweet_id':id,  # "tweet_id" 
        'text':full_text, #"text"
        'airline': airlines, #airline
        'hashtags':entities_hashtag,
        'name':user_name, #"name"
        'user_screen_name':user_screen_name, #"name"???
        'user_location':user_location, #"tweet_location"
        'user_acc_date':user_acc_date,
        'user_verified':user_verified,
        'tweet_coord':coordinates, #"tweet_coord"
        'retweet_count':retweet_count, #"retweet_count"
        'lang':lang}

df = pd.DataFrame(columns)    

In [ ]:
df.to_csv("airline_tweets_nov_25-Dec_02.csv",sep=",",encoding='utf-8')